## Classification of Mexican Restaurants by proximity of tourist places of Medellin

### Dependencies that we will need

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Load the data

### Previous download the data of the site of Medellín and upload into the work directory

#### Load the CVS File in a Pandas DataFrame

In [2]:
with open('MedellinTuristicos.csv') as file1:
    medellin_data = pd.read_csv(file1)

### Show the DataFrame Head

In [3]:
medellin_data.head()

,X,Y,OBJECTID,NOMBRESITIO,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR
0,-75.541587,6.292977,140,Metrocable Santo Domingo Savio - Linea K del M...,Metrocable Santo Domingo Savio,NO,Obras de ingenieria e infraestructura,1,Popular
1,-75.545003,6.287364,147,Unidad de Vida Articulada (UVA) La Esperanza,UVA La Esperanza,NO,Elemento del espacio publico,1,Popular
2,-75.537750,6.295324,146,Unidad de Vida Articulada (UVA) Nuevo Amanecer,UVA Nuevo Amanecer,NO,Elemento del espacio publico,1,Popular
3,-75.543974,6.294812,229,Parque Biblioteca Espana,Parque Biblioteca Espana,NO,Edificacion para la cultura,1,Popular
4,-75.547640,6.294376,230,Parque Lineal la Herrrera,Parque Lineal la Herrrera,NO,Elemento del espacio publico,1,Popular


## Extract the latitude and the longitude of the Medellin City

In [4]:
address = 'Medellin, CO'

geolocator = Nominatim(user_agent="medellin_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Medellin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Medellin are 6.2443382, -75.573553.


### Show the map of the touristic spots

In [5]:
# create map of New York using latitude and longitude values
map_Medellin = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, site in zip(medellin_data['Y'], medellin_data['X'], medellin_data['LABEL']):
    label = '{}'.format(site)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_Medellin)  
    
map_Medellin

### Now define the credentials for Foursquare API

In [6]:
CLIENT_ID = 'W1V4XBKVYDSJPYZONXTZLV0TODADMT5MV5MIQGUR2DAXPVHL' # your Foursquare ID
CLIENT_SECRET = '2BNW013JJV3SODH5V1C0J5SP40HWJQ3E0ZV4DWEPMXMCWCOR' # your Foursquare Secret
VERSION = '20180605' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W1V4XBKVYDSJPYZONXTZLV0TODADMT5MV5MIQGUR2DAXPVHL
CLIENT_SECRET:2BNW013JJV3SODH5V1C0J5SP40HWJQ3E0ZV4DWEPMXMCWCOR


### We need a function to extract the 15 closest restaurants within a radius of 500 meters to each tourist site

In [7]:
def getNearbyRestaurants(names, latitudes, longitudes, radius=500, LIMIT=15):
    
    i = 0
    restaurant_list=[]
    search = 'Mexican'
    for name, lat, lng in zip(names, latitudes, longitudes):
                
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        restaurant_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'].replace("'","")) for v in results])
         

    nearby_restaurant = pd.DataFrame([item for restaurant_list in restaurant_list for item in restaurant_list])
    nearby_restaurant.columns = ['Comuna', 
                  'Comuna Latitude', 
                  'Comuna Longitude', 
                  'Restaurant', 
                  'Restaurant Latitude', 
                  'Restaurant Longitude', 
                  'Restaurant Category']    
    return(nearby_restaurant)

### The dataframe is created with all the restaurants near the tourist places

In [8]:
Medellin_Restaurants = getNearbyRestaurants(names=medellin_data['LABEL'],
                                            latitudes=medellin_data['Y'],
                                            longitudes=medellin_data['X']
                                            )
Medellin_Restaurants.head()

,Comuna,Comuna Latitude,Comuna Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
0,Museo Casa Gardeliana,6.269579,-75.554634,El Monchito,6.270411,-75.557643,Mexican Restaurant
1,Monumento a Carlos Gardel,6.269665,-75.554652,El Monchito,6.270411,-75.557643,Mexican Restaurant
2,CCial. Bosque Plaza,6.268893,-75.565334,Taconazo,6.265693,-75.565063,Burrito Place
3,Ruta N,6.265316,-75.566283,Taconazo,6.265693,-75.565063,Burrito Place
4,Ruta N,6.265316,-75.566283,Comida mexicana del paisa,6.264579,-75.569366,Taco Place


In [20]:
##print(Medellin_Restaurants['Restaurant'].unique())

In [9]:
##print(Medellin_Restaurants.groupby('Restaurant').count())
print('There are {} uniques categories.'.format(len(Medellin_Restaurants['Restaurant Category'].unique())))

There are 6 uniques categories.


In [10]:
##medellin_onehot = pd.get_dummies(Medellin_Restaurants[['Restaurant Category']], prefix="", prefix_sep="")
medellin_onehot = pd.get_dummies(Medellin_Restaurants[['Restaurant']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
medellin_onehot['Comuna'] = Medellin_Restaurants['Comuna'] 

# move neighborhood column to the first column
fixed_columns = [medellin_onehot.columns[-1]] + list(medellin_onehot.columns[:-1])
medellin_onehot = medellin_onehot[fixed_columns]

medellin_onehot.head()
medellin_onehot.shape
medellin_grouped = medellin_onehot.groupby('Comuna').mean().reset_index()

medellin_grouped.head()

num_top_restaurants = 5

for Comuna in medellin_grouped['Comuna']:
    print("----"+Comuna+"----")
    temp = medellin_grouped[medellin_grouped['Comuna'] == Comuna].T.reset_index()
    temp.columns = ['Restaurant','Freq']
    temp = temp.iloc[1:]
    temp['Freq'] = temp['Freq'].astype(float)
    temp = temp.round({'Freq': 2})
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(num_top_restaurants))
    print('\n')

----Academia Antioquena de Historia----
                       Restaurant  Freq
0          Señor Taco Camino Real  0.33
1               The Mexican House  0.33
2       Lupita Peruana & Mexicana  0.33
3  La Bronca - Cantina De Esquina  0.00
4                        Milagros  0.00


----Aeroparque Juan Pablo II----
                  Restaurant  Freq
0  Taquería El Sol de Mexico   1.0
1                 Señor Taco   0.0
2                SIETECHILES   0.0
3    Roll Up - Sushi Burrito   0.0
4        Pick Up Fresh Food!   0.0


----Antiguo Colegio de San Ignacio----
                       Restaurant  Freq
0          Señor Taco Camino Real  0.33
1               The Mexican House  0.33
2       Lupita Peruana & Mexicana  0.33
3  La Bronca - Cantina De Esquina  0.00
4                        Milagros  0.00


----Antiguo Instituto Pedro Justo Berrio y Parroquia San Juan Bosco----
               Restaurant  Freq
0  Ketaco Comida Mexicana   0.5
1              Don Aníbal   0.5
2                ¡Ay Wey

## Mexican Restaurants nearby the touristics spots.

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Comuna']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
Medellin_Restaurants_sorted = pd.DataFrame(columns=columns)
Medellin_Restaurants_sorted['Comuna'] = medellin_grouped['Comuna']

for ind in np.arange(medellin_grouped.shape[0]):
    Medellin_Restaurants_sorted.iloc[ind, 1:] = return_most_common_venues(medellin_grouped.iloc[ind, :], num_top_venues)

Medellin_Restaurants_sorted.head()

,Comuna,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
0,Academia Antioquena de Historia,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
1,Aeroparque Juan Pablo II,Taquería El Sol de Mexico,Órale,El Jitomate,Donde Hernán,Eat Burrito
2,Antiguo Colegio de San Ignacio,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
3,Antiguo Instituto Pedro Justo Berrio y Parroqu...,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
4,Asociacion Pequeno Teatro,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería


## Apply the kmeans algorithm

In [13]:
# set number of clusters
kclusters = 5

medellin_grouped_clustering = medellin_grouped.drop('Comuna', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(medellin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 4, 3, 3, 3, 4, 3, 4], dtype=int32)

In [14]:
Medellin_Restaurants_sorted.head()

,Comuna,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
0,Academia Antioquena de Historia,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
1,Aeroparque Juan Pablo II,Taquería El Sol de Mexico,Órale,El Jitomate,Donde Hernán,Eat Burrito
2,Antiguo Colegio de San Ignacio,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
3,Antiguo Instituto Pedro Justo Berrio y Parroqu...,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
4,Asociacion Pequeno Teatro,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería


In [15]:
Medellin_Restaurants_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
medellin_merged = medellin_data
medellin_merged = medellin_merged.join(Medellin_Restaurants_sorted.set_index('Comuna'), on='LABEL', how='inner')
##medellin_merged.head() # check the last columns!

## Show the dataset's head of the cluster results

In [16]:
medellin_merged.head()


,X,Y,OBJECTID,NOMBRESITIO,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
5,-75.554634,6.269579,221,Museo Casa Gardeliana,Museo Casa Gardeliana,NO,Edificacion para la cultura,3,Manrique,0,El Monchito,Órale,El Jitomate,Donde Hernán,Eat Burrito
6,-75.554652,6.269665,227,Monumento a Carlos Gardel,Monumento a Carlos Gardel,NO,Estatua,3,Manrique,0,El Monchito,Órale,El Jitomate,Donde Hernán,Eat Burrito
11,-75.565334,6.268893,24,Centro Comercial Bosque Plaza,CCial. Bosque Plaza,NO,Arquitectura para el comercio,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
13,-75.566283,6.265316,62,Ruta N,Ruta N,NO,Edificacion para la ciencia,4,Aranjuez,0,Taconazo,Comidas Mexicanas de la U.de.A,Comida mexicana del paisa,Los Burritos del Coste,Órale
14,-75.568775,6.267040,63,Universidad de Antioquia,Universidad de Antioquia,NO,Edificacion para la educacion,4,Aranjuez,0,Taconazo,Comidas Mexicanas de la U.de.A,Comida mexicana del paisa,Los Burritos del Coste,Órale


## Show the visualitation of the clusters

In [17]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

##print(medellin_merged)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(medellin_merged['Y'], medellin_merged['X'], medellin_merged['LABEL'], medellin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Show the cluster zero.

In [25]:
medellin_merged.loc[medellin_merged['Cluster Labels'] == 0, medellin_merged.columns[[1] + list(range(4, medellin_merged.shape[1]))]]

,Y,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
5,6.269579,Museo Casa Gardeliana,NO,Edificacion para la cultura,3,Manrique,0,El Monchito,Órale,El Jitomate,Donde Hernán,Eat Burrito
6,6.269665,Monumento a Carlos Gardel,NO,Estatua,3,Manrique,0,El Monchito,Órale,El Jitomate,Donde Hernán,Eat Burrito
13,6.265316,Ruta N,NO,Edificacion para la ciencia,4,Aranjuez,0,Taconazo,Comidas Mexicanas de la U.de.A,Comida mexicana del paisa,Los Burritos del Coste,Órale
14,6.267040,Universidad de Antioquia,NO,Edificacion para la educacion,4,Aranjuez,0,Taconazo,Comidas Mexicanas de la U.de.A,Comida mexicana del paisa,Los Burritos del Coste,Órale
25,6.270566,Parque Comercial Florida,NO,Arquitectura para el comercio,5,Castilla,0,Señor Taco,Ay Caramba!,El Jitomate,Donde Hernán,Eat Burrito
43,6.240497,Cristo El Salvador,NO,Estatua,9,Buenos Aires,0,Ketako,Órale,El Jitomate,Donde Hernán,Eat Burrito
44,6.240603,Cerro El Salvador,NO,Cerro,9,Buenos Aires,0,Ketako,Órale,El Jitomate,Donde Hernán,Eat Burrito
57,6.234528,Hospital General de Medellin,NO,Centro hospitalario y asistencial,10,La Candelaria,0,Tacos&Bar-BQ,El Taconazo de Texaco Juanbe,El Jitomate,Donde Hernán,Eat Burrito
58,6.236359,Iglesia Nuestra Senora del Perpetuo Socorro,NO,Templo,10,La Candelaria,0,El Taconazo de Texaco Juanbe,Órale,El Jitomate,Donde Hernán,Eat Burrito
59,6.228608,CCial. Premium Plaza,NO,Arquitectura para el comercio,10,La Candelaria,0,El Cuate Delicias Mexicanas,El Patio de Lupe,Barrita Burrito,Tacos & Bar-BQ,Órale


## Show the cluster one.

In [26]:
medellin_merged.loc[medellin_merged['Cluster Labels'] == 1, medellin_merged.columns[[1] + list(range(4, medellin_merged.shape[1]))]]

,Y,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
56,6.252909,Casa Sede Montesacro ? PREVER,NO,Funeraria,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
70,6.252792,Parque Bolivar,NO,Elemento del espacio publico,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
76,6.251351,Ed. Naviera Colombiana,NO,Edificacion para la educacion,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
77,6.250828,Ed. Palace,NO,Arquitectura para el comercio,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
97,6.251381,Plazuela Nutibara,NO,Elemento del espacio publico,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
99,6.250068,Parque de Berrio,NO,Elemento del espacio publico,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
129,6.252116,Plaza Botero,SI,Elemento del espacio publico,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
130,6.250087,Iglesia La Candelaria,SI,Templo,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
132,6.251754,Palacio de la Cultura,SI,Edificacion para la cultura,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito
133,6.252036,Hotel Nutibara,SI,Arquitectura para el comercio,10,La Candelaria,1,Señor Taco Camino Real,Delirio Exquisito,Don Aníbal,Donde Hernán,Eat Burrito


## Show the cluster two.

In [28]:
medellin_merged.loc[medellin_merged['Cluster Labels'] == 2, medellin_merged.columns[[1] + list(range(4, medellin_merged.shape[1]))]]

,Y,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
11,6.268893,CCial. Bosque Plaza,NO,Arquitectura para el comercio,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
15,6.268117,Casa de la Musica,NO,Edificacion para la cultura,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
17,6.268292,Esquina de las Mujeres,NO,Elemento del espacio publico,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
20,6.268345,Parque de los Deseos,SI,Escenario para la recreacion y el deporte,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
21,6.268650,Planetario,SI,Edificacion para la ciencia,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
23,6.265735,Museo Cementerio San Pedro,SI,Funeraria,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito
24,6.269402,Jardin Botanico,SI,Edificacion para la ciencia,4,Aranjuez,2,Taconazo,Órale,Delirio Exquisito,Donde Hernán,Eat Burrito


## show the cluster three.

In [29]:
medellin_merged.loc[medellin_merged['Cluster Labels'] == 3, medellin_merged.columns[[1] + list(range(4, medellin_merged.shape[1]))]]

,Y,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
52,6.250931,Casa Sede Banco de Colombia,NO,Arquitectura para el comercio,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
54,6.246527,CCial. El Punto de La Oriental,NO,Arquitectura para el comercio,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Órale,El Jitomate,Donde Hernán
61,6.245732,Placita de Florez,NO,Arquitectura para el comercio,10,La Candelaria,3,The Mexican House,Lupita Peruana & Mexicana,Órale,El Jitomate,Donde Hernán
62,6.247602,Teatro Pablo Tobon Uribe,NO,Edificacion para la cultura,10,La Candelaria,3,The Mexican House,Lupita Peruana & Mexicana,Órale,El Jitomate,Donde Hernán
63,6.248263,Palacio de Bellas Artes,NO,Edificacion para la cultura,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
64,6.247554,Asociacion Pequeno Teatro,NO,Edificacion para la cultura,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
69,6.247277,Tranvia de Ayacucho,NO,Obras de ingenieria e infraestructura,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
71,6.248374,Avenida La Playa,NO,Elemento del espacio publico,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
72,6.246506,Ed. Paraninfo Universidad de Antioquia,NO,Edificacion para la cultura,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería
73,6.246019,Antiguo Colegio de San Ignacio,NO,Edificacion para la educacion,10,La Candelaria,3,Señor Taco Camino Real,The Mexican House,Lupita Peruana & Mexicana,Órale,El Güero Taquería


## Show the cluster four.

In [30]:
medellin_merged.loc[medellin_merged['Cluster Labels'] == 4, medellin_merged.columns[[1] + list(range(4, medellin_merged.shape[1]))]]

,Y,LABEL,IMPERDIBLE,TIPOATRACTIVO,COD_COMUNA,COM_CORR,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant
46,6.248383,Zona Comercial del Hueco,NO,Arquitectura para el comercio,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
47,6.247542,CCial. Gran Plaza,NO,Arquitectura para el comercio,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
49,6.245423,Parque San Antonio,NO,Elemento del espacio publico,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
50,6.245022,Museo del Agua EPM,NO,Edificacion para la cultura,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
67,6.247222,Metro de Medellin,NO,Obras de ingenieria e infraestructura,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
68,6.251100,Iglesia San Juan de Dios,NO,Templo,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
74,6.249451,CCial. Palacio Nacional,NO,Arquitectura para el comercio,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
100,6.249362,Banco de La Republica,NO,Edificacion para la cultura,10,La Candelaria,4,Don Aníbal,Ketaco Comida Mexicana,Señor Taco Camino Real,El Jitomate,Donde Hernán
105,6.244499,Parroquia San Antonio,NO,Templo,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
108,6.244607,Plaza de la Libertad,NO,Elemento del espacio publico,10,La Candelaria,4,Ketaco Comida Mexicana,Don Aníbal,El Viejo y El Pancake,El Trio,Donde Hernán
